<a href="https://colab.research.google.com/github/tintenderete/TFM/blob/main/individuos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# class Creacion_inidividuos():

In [446]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import ConvLSTM1D, Dense, Conv2D, LSTM, GRU, Reshape, ConvLSTM2D, Conv3D, TimeDistributed, Flatten
import keras
import tensorflow as tf
class Creacion_inidividuos():

  def __init__(self):
    self.a = 0

  def get_activation_string(self,type_activation):
    if type_activation == 0:
      return 'tanh'
    else:
      return 'relu'
  
  # # # LAYERS # # #

  
  def new_dense_layer(self, info):
      return Dense(activation=self.get_activation_string(info[0]), units = info[1] )

  def new_timeDistributed_dense(self, info):
    return TimeDistributed(Dense(activation=info[0], units = info[1] ))
    
  # layers 4 -

  def new_convlstm2d_layer(self, info):
    return ConvLSTM2D(activation=info[0],filters=info[1], kernel_size=info[2])

  def new_conv3d_layer(self, info):
    return Conv3D(activation=info[0], filters= info[1], kernel_size = info[2]  )

  # layers 3 - 3

  def new_lstm1d_layer(self, info):
    return ConvLSTM1D(activation = self.get_activation_string(info[0]),filters=info[1],kernel_size = info[2])

  def new_conv2d_layer(self, info):
    return Conv2D(activation = self.get_activation_string(info[0]), filters = info[1], kernel_size = info[2])

  #layers 3 - 2
  def new_reshape_layer(self, shape):
    return Reshape((shape[0],shape[1]*shape[2]))

  # layers 2 - 1

  def new_lstm_layer(self, info):
    return LSTM( units =  info[1])

  def new_gru_layer(self, info):
    return GRU(units = info[1])

  # # # END LAYERS # # #


  def get_layer(self, layer_type, info):
    if  layer_type == 0:
      return               (     Dense(                            activation=info[0],   units=info[1] )                                          ,-1)   # xD
    elif layer_type == 1:
      return                (     TimeDistributed(Dense(            activation=info[0],   units=info[1] ))                                        , 4)  # Menos de 3 no funciona
    elif layer_type == 2:
      return               (     ConvLSTM2D(                       activation=info[0],   filters=info[1],   kernel_size=info[2],  padding='same') ,5)# 5D
    elif layer_type == 3:
      return                (    Conv3D(                           activation=info[0],   filters=info[1],   kernel_size=info[2],  padding='same')                  ,5)# 5D
    elif layer_type == 4:
      return                (     ConvLSTM1D(                       activation=info[0],   filters=info[1],   kernel_size=info[2] ,  padding='same')               ,4)# 4D
    elif layer_type == 5:
      return                 (    Conv2D(                           activation=info[0],   filters=info[1],   kernel_size=info[2] ,  padding='same')              ,4)# 4D 
    elif layer_type == 6:
      return                 (    LSTM(                                                   units=info[1])                                         ,3) # 3D
    elif layer_type == 7:
      return                  (   GRU(                                                    units=info[1])                                          ,3)# 3D
    


  def get_layer_3(self, layer_type= 0, info = ["tanh",0,0], shape = (1,1,1)):

    if layer_type == 0:
      return self.new_dense_layer(info)
    elif layer_type == 1 :
      return self.new_conv2d_layer(info)
    elif layer_type == 2 :
      return self.new_reshape_layer(shape)

    return print("fail get_layer_3")


  def get_layer_2(self, layer_type= 0, info = ["tanh",0,0] , shape = (1,1,1)):
    
    if layer_type == 0:
      return self.new_dense_layer(info)
    elif layer_type == 1:
      return self.new_lstm_layer(info)
    elif layer_type == 2 :
      return self.new_gru_layer(info)

crear = Creacion_inidividuos()

In [465]:
def aumenta_dimensions(model):
  return tuple( tf.expand_dims(model, axis=-1).shape[1:].as_list() )

In [466]:
def reduce_dimensions(shape):
  d = list(shape[1:])
  new_shape = d[:-2] + [d[-2] * d[-1]]
  return tuple(new_shape)

# ---- INDIVIDUOS 2 ---- 

In [381]:
info =  ["tanh",3,2]

In [382]:
x = np.random.rand(100, 60, 35, 3)
y = np.random.rand(100, 35)

In [467]:
# pongo -1 para que todos los individuos no tengan el mismo numero de capas, si sale -1 se corta la cadena
individuos = np.random.randint(0, 8, (10, 10))

In [468]:
ind = individuos[0]
ind

array([1, 5, 2, 2, 3, 7, 4, 7, 1, 3])

In [470]:
for ind in individuos:

  inputs = keras.Input(shape=(x.shape[1:]))
  m = inputs

  for gen in ind:
    #gen = ind[count]
    #count += 1

    if gen != -1:

      #print("count: " + str(count) + " gen: " + str(gen))
      new_layer , dim_layer = crear.get_layer(gen, info)

      current_m_dim = len(m.shape) 
      new_layer_dim = current_m_dim if dim_layer == -1 else dim_layer 
      diferencia = new_layer_dim - current_m_dim

      print("---------------------")
      print("current_m_dim: " +  str(m.shape))
      print("new_layer_name: " + str(new_layer.name) + " dim: " + str(diferencia))
      
      

      while diferencia != 0:
        
        if diferencia > 0:
          #m = Reshape( tuple( tf.expand_dims(m, axis=-1).shape[1:].as_list() ) ) (m)
          new_d = aumenta_dimensions(m)
          m = Reshape(new_d)(m)
        elif diferencia < 0:
          new_d = reduce_dimensions(m.shape)
          m = Reshape(new_d)(m)
          
        
        current_m_dim = len(m.shape) 
        new_layer_dim = current_m_dim if dim_layer == -1 else dim_layer 
        diferencia = new_layer_dim - current_m_dim

        print("fixed_new_m_dim: " +  str(m.shape))

      m = new_layer(m)

      print("new_m_dim: " +  str(m.shape))
    
    else:
      break


  m = Flatten()(m)
  outputs = Dense(y.shape[1])(m)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
    loss='mse',
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
  )

---------------------
current_m_dim: (None, 60, 35, 3)
new_layer_name: time_distributed_271 dim: 0
new_m_dim: (None, 60, 35, 3)
---------------------
current_m_dim: (None, 60, 35, 3)
new_layer_name: conv2d_266 dim: 0
new_m_dim: (None, 60, 35, 3)
---------------------
current_m_dim: (None, 60, 35, 3)
new_layer_name: conv_lstm2d_290 dim: 1
fixed_new_m_dim: (None, 60, 35, 3, 1)
new_m_dim: (None, 35, 3, 3)
---------------------
current_m_dim: (None, 35, 3, 3)
new_layer_name: conv_lstm2d_291 dim: 1
fixed_new_m_dim: (None, 35, 3, 3, 1)
new_m_dim: (None, 3, 3, 3)
---------------------
current_m_dim: (None, 3, 3, 3)
new_layer_name: conv3d_243 dim: 1
fixed_new_m_dim: (None, 3, 3, 3, 1)
new_m_dim: (None, 3, 3, 3, 3)
---------------------
current_m_dim: (None, 3, 3, 3, 3)
new_layer_name: gru_240 dim: -2
fixed_new_m_dim: (None, 3, 3, 9)
fixed_new_m_dim: (None, 3, 27)
new_m_dim: (None, 3)
---------------------
current_m_dim: (None, 3)
new_layer_name: conv_lstm1d_287 dim: 2
fixed_new_m_dim: (None, 3